In [7]:
import pandas as pd
import numpy as np

def read_csv(file_path):
    """Read CSV file into a DataFrame."""
    return pd.read_csv(file_path, header=None)

def initialize_sets(features):
    """Initialize the General (G) and Specific (S) sets."""
    num_attributes = features.shape[1]
    specific = ['0'] * num_attributes  # Most specific hypothesis
    general = [['?' for _ in range(num_attributes)] for _ in range(num_attributes + 1)]  # Most general hypotheses
    general[0] = ['0'] * num_attributes  # Initial general hypothesis
    return specific, general

def update_specific(specific, example):
    """Update the Specific set based on the positive example."""
    for i in range(len(specific)):
        if specific[i] == '0' or specific[i] == example[i]:
            specific[i] = example[i]
        else:
            specific[i] = '?'
    return specific

def update_general(general, specific, example):
    """Update the General set based on the positive example."""
    new_general = [g for g in general if is_more_general(g, specific)]
    
    specific_copy = specific[:]
    for i in range(len(specific)):
        if specific[i] == '?':
            continue
        temp_general = specific_copy[:]
        temp_general[i] = '?'
        if not any(np.array_equal(temp_general, np.array(g)) for g in new_general):
            new_general.append(temp_general)
    
    return new_general

def is_more_general(g, s):
    """Check if a hypothesis g is more general than hypothesis s."""
    return all((g_i == '?' or g_i == s_i) for g_i, s_i in zip(g, s))

def candidate_elimination_algorithm(data):
    """Apply the Candidate-Elimination algorithm to the training data."""
    features = data.iloc[:, :-1].values
    target = data.iloc[:, -1].values

    specific, general = initialize_sets(features)

    for i, example in enumerate(features):
        if target[i].strip().lower() == 'yes':
            print(f"Processing positive example {example} with target {target[i]}")
            specific = update_specific(specific, example)
            print("Updated Specific:", specific)
            general = update_general(general, specific, example)
            print("Updated General:")
            for g in general:
                print(g)
        else:
            general = [g for g in general if not np.array_equal(g, specific)]
    
    # Remove redundant general hypotheses that are more specific than others
    final_general = []
    for g in general:
        if not any(is_more_general(g2, g) for g2 in final_general):
            final_general.append(g)
    
    return specific, final_general

def main():
    """Main function to read data and apply Candidate-Elimination algorithm."""
    file_path = 'training_data.csv'  # Define the path to your CSV file
    data = read_csv(file_path)
    specific, general = candidate_elimination_algorithm(data)
    
    print("Final Specific_h:")
    print(np.array(specific))  # Output as numpy array for clarity
    print("Final General_h:")
    for g in general:
        print(np.array(g))  # Output as numpy array for clarity

if __name__ == "__main__":
    main()


Processing positive example ['Sunny' ' Warm' ' Normal' ' Strong' ' Warm' ' Same'] with target  Yes
Updated Specific: ['Sunny', ' Warm', ' Normal', ' Strong', ' Warm', ' Same']
Updated General:
['?', '?', '?', '?', '?', '?']
['?', '?', '?', '?', '?', '?']
['?', '?', '?', '?', '?', '?']
['?', '?', '?', '?', '?', '?']
['?', '?', '?', '?', '?', '?']
['?', '?', '?', '?', '?', '?']
['?', ' Warm', ' Normal', ' Strong', ' Warm', ' Same']
['Sunny', '?', ' Normal', ' Strong', ' Warm', ' Same']
['Sunny', ' Warm', '?', ' Strong', ' Warm', ' Same']
['Sunny', ' Warm', ' Normal', '?', ' Warm', ' Same']
['Sunny', ' Warm', ' Normal', ' Strong', '?', ' Same']
['Sunny', ' Warm', ' Normal', ' Strong', ' Warm', '?']
Processing positive example ['Sunny' ' Warm' ' High' ' Strong' ' Warm' ' Same'] with target  Yes
Updated Specific: ['Sunny', ' Warm', '?', ' Strong', ' Warm', ' Same']
Updated General:
['?', '?', '?', '?', '?', '?']
['?', '?', '?', '?', '?', '?']
['?', '?', '?', '?', '?', '?']
['?', '?', '?', '